In [5]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [2]:
response = requests.get('https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-139.219,-31.354,135,66.513&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=2')

response.status_code

200

Tried accessing the api, but do not have authentication

In [48]:
headers = { 
   'authority': '0nn8jwscqe.execute-api.us-east-1.amazonaws.com',
   'method': 'GET', 
   'path': '/prod/fm-elastic-prod-cds?queryType=list&view=fm-view-list&subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-139.219,-31.354,135,66.513&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=gm&windRoseAnd=undefined&zoom=2&listType=gm&sortColumn=amount&sortOrder=desc',
   'scheme': 'https',
  'accept': 'application/json, text/javascript, */*; q=0.01',
  'accept-encoding': 'gzip, deflate, br, zstd',
  'accept-language': 'en-US,en;q=0.9',
  'origin': 'https://maps.foundationcenter.org',
  'authentication': '',
  'content-type': 'application/json; charset=utf-8',
  'cookie': 'rdpirm01=',
  'dnt': '0',
  'referer': 'https://maps.foundationcenter.org/',
  'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': "macOS",
  'sec-fetch-dest': 'empty',                        
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
}

response = requests.get('https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-133.9453125,14.689881366618762,-37.6171875,55.825973254619015&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=4')
response.status_code

  # Parse the content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract data based on the provided selectors
grants_data = []

# Locate all rows of grant data (assuming each row has the required fields)
rows = soup.select("table tbody tr")  # Adjust if necessary for table structure
for row in rows:
    funder_name = row.select_one("a.gm-link").get_text(strip=True) if row.select_one("a.gm-link") else None
    recipient = row.select_one("a.recip-link").get_text(strip=True) if row.select_one("a.recip-link") else None
    year = row.select_one("td.cell-year").get_text(strip=True) if row.select_one("td.cell-year") else None
    value = row.select_one("td.sorting_2").get_text(strip=True) if row.select_one("td.sorting_2") else None

    # Append to the list if data is found
    grants_data.append({
        "funder_name": funder_name,
        "recipient": recipient,
        "year": year,
        "value": value
    })

# response.content
# response.headers

Use Playwright

In [52]:
import asyncio
from playwright.async_api import async_playwright

async def download_csv():
    async with async_playwright() as p:
        # Launch the browser (set headless=False if you want to see the browser actions)
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        
        # Specify the download path for Playwright to store files
        # context.set_default_download_path("/Users/yanans/Desktop/donor/")  # Set your download path

        # Open a new page
        page = await context.new_page()

        # Go to the target URL
        await page.goto("https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-133.9453125,14.689881366618762,-37.6171875,55.825973254619015&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=4")

        # Loop through the dynamic views
        for i in range(1, 6):  # Assume 5 dynamic views/pages to navigate through
            # Perform actions to navigate to the next view (e.g., clicking a "Next" button)
            # Adjust selector to match the button for switching pages
            # await page.click("button#next-page")

            # await page.get_by_text(f'{i}').click()

            # pagi = page.locator('.pagination')
            # await locator.get_by_text(f'{i}').click()
            next_page = i + 1 
            # await page.click("ul.pagination li a:has-text('1')")


            # Wait for the CSV download button to be ready
            # locator = page.locator(".export-csv")
            # locator = page.locator(".widget-export-tools > span:nth-child(2)").first()
            # locator = page.locator("#profile-modal #fm-pdf-csv-2 span").nth(1)
            # 2) <span class="spriteimg export-csv"></span> aka locator("#modal-all-list #fm-pdf-csv-2 span").nth(1)
            # 3) <span class="spriteimg export-csv"></span> aka locator("#profile-modal #fm-pdf-csv-2 span").nth(1)
            # 4) <span class="spriteimg export-csv"></span> aka locator("#profile-modal-dashboard #fm-pdf-csv-2 span").nth(1)
            # 5) <span class="spriteimg export-csv"></span> aka locator("#regional-modal-dashboard #fm-pdf-csv-2 span").nth(1)
            # locator.wait_for(state="visible")
            # await locator.click()
            # await page.wait_for_selector("span#spriteimg export-csv")  # Adjust selector as needed

            # grant = page.locator("li.grants-link a#list-screen-all")
            # if await page.locator("li.grants-link a#list-screen-all"):
            await page.click("li.grants-link")


            # Trigger download and wait for it to complete
            # download = await page.wait_for_event("download", lambda download: download.url.endswith(".csv"))
            # await download.save_as("/Users/yanans/Desktop/donor/" + f"data_view_{i}.csv")

            print(f"Downloaded CSV for view {i}")
            break

        # Close the browser
        await browser.close()

# Run the async function
# asyncio.run(download_csv())
# RuntimeError: asyncio.run() cannot be called from a running event loop
await download_csv()


TimeoutError: Page.click: Timeout 30000ms exceeded.
Call log:
waiting for locator("li.grants-link")
  -   locator resolved to 2 elements. Proceeding with the first one: <li class="grants-link">…</li>
  - attempting click action
  -   waiting for element to be visible, enabled and stable
  -   element is not visible
  - retrying click action, attempt #1
  -   waiting for element to be visible, enabled and stable
  -   element is not stable
  - retrying click action, attempt #2
  -   waiting 20ms
  -   waiting for element to be visible, enabled and stable
  -   element is visible, enabled and stable
  -   scrolling into view if needed
  -   done scrolling
  -   <li>…</li> from <div id="welcome-modal" aria-hidden="false" class="modal fade in">…</div> subtree intercepts pointer events
  - retrying click action, attempt #3
  -   waiting 100ms
  -   waiting for element to be visible, enabled and stable
  -   element is visible, enabled and stable
  -   scrolling into view if needed
  -   don

Use selenium

In [9]:
# Import Module
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

 
# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
# driver = webdriver.Chrome()
 
# Open URL
driver.get(
    'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8')
 
# Start Searching
try:
    driver.find_element(By.ID, "fm-start-searching").click()
    # driver.find_element(By.CLASS_NAME, "btn btn-primary").click()
except Exception as e:
    print(e)
else:
    # driver.find_element(By.CLASS_NAME, "btn btn-primary")
    i = 1
    while i<3:
        # Click "Grants" button
        driver.find_element(By.CLASS_NAME, 'grants-link').click()
    
        # Click "Download this data" button
        label = driver.find_element(By.CLASS_NAME, "widget-export-tools")
        try:
            # label.find_element(By.CLASS_NAME, 'spriteimg.export-csv')
            label.find_element(By.CLASS_NAME, 'spriteimg.export-csv').click()
        except Exception as e:
            print(e)
        else:
            print(f"Button clicked for page{i+1}")
            page = driver.find_element(By.CLASS_NAME, 'table-pagination').find_element(By.CLASS_NAME, 'pagination').find_element(By.CLASS_NAME, 'active')
            next_page = await page.find_element(By.XPATH,"following-sibling::li").click()
            i+=1
            # driver.find_element(By.XPATH, '//*[@id="fm-list-1grantfm-list-table_wrapper"]/div[2]/div[2]/div/ul/li[2]/following-sibling::li').find_element(By.XPATH, '/a').click()

driver.quit()
# Click on Download Button
# driver.find_element_by_id('link-to-download').click()

Button clicked for page2


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=129.0.6668.101)
Stacktrace:
0   chromedriver                        0x0000000101294500 cxxbridge1$str$ptr + 1917112
1   chromedriver                        0x000000010128c890 cxxbridge1$str$ptr + 1885256
2   chromedriver                        0x0000000100e9c3a8 cxxbridge1$string$len + 89024
3   chromedriver                        0x0000000100ee1838 cxxbridge1$string$len + 372816
4   chromedriver                        0x0000000100ed727c cxxbridge1$string$len + 330388
5   chromedriver                        0x0000000100ed6d18 cxxbridge1$string$len + 329008
6   chromedriver                        0x0000000100f1ab7c cxxbridge1$string$len + 607124
7   chromedriver                        0x0000000100ed5374 cxxbridge1$string$len + 322444
8   chromedriver                        0x0000000100ed5fc4 cxxbridge1$string$len + 325596
9   chromedriver                        0x000000010125bd2c cxxbridge1$str$pt

In [ ]:
https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-139.219,-31.354,135,66.513&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=2

In [6]:
import pandas as pd
import glob
import os

# Set the path to the folder containing CSV files
folder_path = 'Grants'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and concatenate all CSV files
df_list = [pd.read_csv(file) for file in csv_files]
concatenated_df = pd.concat(df_list, ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
output_path = 'grants_info.csv'
concatenated_df.to_csv(output_path, index=False)

print(f"Concatenated CSV file saved to {output_path}")


ParserError: Error tokenizing data. C error: Expected 1 fields in line 7, saw 4


Find 2024 grants details

In [132]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Store data
funder, funder_addr, recipient, recipient_addr, amount, year, area, duration, title, subject, population, strategy, transaction, desc, source = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
# Open browser
url = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2024&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8'

# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(url)

# Start Searching
try:
    driver.find_element(By.ID, "fm-start-searching").click()
    # driver.find_element(By.CLASS_NAME, "btn btn-primary").click()
except Exception as e:
    print(e)
else:
    # storing the current window handle to get back to dashboard
    main_page = driver.current_window_handle

    # Click "Grants"
    grants = driver.find_element(By.CLASS_NAME, 'grants-link')
    grants.click()
    
    time.sleep(2)
    # Locate a row
    all_list = driver.find_element(By.ID, 'list-screen-all')    #<div class="tab-pane fade active in" id="list-screen-all">
    tbody = all_list.find_element(By.TAG_NAME, 'tbody')         # <tbody>
    # row = driver.find_element(By.XPATH, '//*[@id="fm-list-1grantfm-list-table"]/tbody/tr[1]')
    row = tbody.find_element(By.CLASS_NAME, 'gradeX')
    # row = driver.find_elements(By.CLASS_NAME, 'gradeX')

    
    order = 0
    # Each page has 100 rows
    for i in range(100):
        # time.sleep(2)

        # wait = WebDriverWait(driver, 20)
        # Click each button
        # detail_button = WebDriverWait(row, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "table-detail-icon")))

        # Press button
        row.find_element(By.CLASS_NAME, "table-detail-icon").click()
        # detail_button = driver.find_elements(By.CLASS_NAME, "table-detail-icon")[i]
        # try:
            # detail_button.click()
        # except Exception as e:
        #     # element = WebDriverWait(row, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "table-detail-icon")))
        #     # element.click()
        #     row.find_element(By.CLASS_NAME, "table-detail-icon").click()

        # Wait for the window to pop up
        wait = WebDriverWait(driver, timeout=2)
        window = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="fm-list-1popupprofile-modal-new"]')))

        # Collect details
        amount.append(window.find_element(By.CLASS_NAME, "modal-grant-details").find_element(By.ID, "grant-detail-total-amount").text)
        funder.append(window.find_element(By.ID, "grant-detail-grantmaker").text)
        funder_addr.append(window.find_element(By.ID, "grant-detail-gm-address").text)
        recipient.append(window.find_element(By.ID, "grant-detail-recipient").text)
        recipient_addr.append(window.find_element(By.ID, "grant-detail-recipient-address").text)
        year.append(window.find_element(By.ID, "grant-detail-year").text)
        # print(year)
        area.append(window.find_element(By.ID, 'grant-detail-program-area').text)
        # print(area)
        duration.append(window.find_element(By.ID, 'grant-detail-duration').text)
        # print(duration)
        title.append(window.find_element(By.ID, 'grant-detail-title').text)
        # print(title)
        subject.append(window.find_element(By.ID, 'grant-detail-subject').text)
        # print(subjects)
        population.append(window.find_element(By.ID, 'grant-detail-population').text)
        # print(population)
        strategy.append(window.find_element(By.ID, 'grant-detail-strategy-type').text)
        # print(strategy)
        transaction.append(window.find_element(By.ID, 'grant-detail-transaction-type').text)
        # print(type)        
        desc.append(window.find_element(By.ID, 'grant-detail-description').text)
        source.append(window.find_element(By.ID, 'data-license').find_element(By.TAG_NAME, "a").get_attribute('href'))
        
        window.find_element(By.CLASS_NAME, 'close').click()

        # Find next sibling row until done
        time.sleep(2)
        try:
            row = row.find_element(By.XPATH,"following-sibling::tr")
        except Exception as e:
            print(e)
        break
            

driver.quit()



In [20]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Store data
funder, funder_addr, recipient, recipient_addr, amount, year, area, duration, title, subject, population, strategy, transaction, desc, source = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
# Open browser
url = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2024&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8'

# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(url)

# Start Searching
try:
    driver.find_element(By.ID, "fm-start-searching").click()
    # driver.find_element(By.CLASS_NAME, "btn btn-primary").click()
except Exception as e:
    print(e)
else:
    # storing the current window handle to get back to dashboard
    main_page = driver.current_window_handle

    # Click "Grants"
    grants = driver.find_element(By.CLASS_NAME, 'grants-link')
    grants.click()
    
    time.sleep(2)
    
    # Start from page 1
    num = 1
    while num <=3:
        
        # # Locate a row
        all_list = driver.find_element(By.ID, 'list-screen-all')    #<div class="tab-pane fade active in" id="list-screen-all">
        tbody = all_list.find_element(By.TAG_NAME, 'tbody')         # <tbody>
        row = tbody.find_element(By.CLASS_NAME, 'gradeX')
        
        # Each page has 100 rows
        for i in range(100):

            # Press button
            row.find_element(By.CLASS_NAME, "table-detail-icon").click()

            # Wait for the window to pop up
            wait = WebDriverWait(driver, timeout=2)
            window = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="fm-list-1popupprofile-modal-new"]')))

            # Collect details
            amount.append(window.find_element(By.CLASS_NAME, "modal-grant-details").find_element(By.ID, "grant-detail-total-amount").text)
            funder.append(window.find_element(By.ID, "grant-detail-grantmaker").text)
            funder_addr.append(window.find_element(By.ID, "grant-detail-gm-address").text)
            recipient.append(window.find_element(By.ID, "grant-detail-recipient").text)
            recipient_addr.append(window.find_element(By.ID, "grant-detail-recipient-address").text)
            year.append(window.find_element(By.ID, "grant-detail-year").text)
            # print(year)
            area.append(window.find_element(By.ID, 'grant-detail-program-area').text)
            # print(area)
            duration.append(window.find_element(By.ID, 'grant-detail-duration').text)
            # print(duration)
            title.append(window.find_element(By.ID, 'grant-detail-title').text)
            # print(title)
            subject.append(window.find_element(By.ID, 'grant-detail-subject').text)
            # print(subjects)
            population.append(window.find_element(By.ID, 'grant-detail-population').text)
            # print(population)
            strategy.append(window.find_element(By.ID, 'grant-detail-strategy-type').text)
            # print(strategy)
            transaction.append(window.find_element(By.ID, 'grant-detail-transaction-type').text)
            # print(type)        
            desc.append(window.find_element(By.ID, 'grant-detail-description').text)
            source.append(window.find_element(By.ID, 'data-license').find_element(By.TAG_NAME, "a").get_attribute('href'))
            
            window.find_element(By.CLASS_NAME, 'close').click()

            # Find next sibling row until done
            time.sleep(1)
            try:
                row = row.find_element(By.XPATH,"following-sibling::tr")
            except Exception as e:
                print(e)

        print(f"Finished page {num}")
        if num == 3:
            break
        else:
            time.sleep(1)

            # Find the active page
            page = all_list.find_element(By.CLASS_NAME, 'table-pagination').find_element(By.CLASS_NAME, 'pagination').find_element(By.CLASS_NAME, "active").find_element(By.XPATH,"following-sibling::li")
            
            # Click the next page
            next_page = WebDriverWait(page, 10).until(EC.element_to_be_clickable((By.TAG_NAME,"a"))).click()

            num+=1
            time.sleep(1)

            

driver.quit()



Message: no such element: Unable to locate element: {"method":"xpath","selector":"following-sibling::tr"}
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010451c500 cxxbridge1$str$ptr + 1917112
1   chromedriver                        0x0000000104514890 cxxbridge1$str$ptr + 1885256
2   chromedriver                        0x0000000104124538 cxxbridge1$string$len + 89424
3   chromedriver                        0x0000000104168878 cxxbridge1$string$len + 368784
4   chromedriver                        0x000000010415ed18 cxxbridge1$string$len + 329008
5   chromedriver                        0x00000001041a2b7c cxxbridge1$string$len + 607124
6   chromedriver                        0x000000010415d374 cxxbridge1$string$len + 322444
7   chromedriver                        0x000000010415dfc4 cxxbridge1$s

In [22]:
import pandas as pd

funder = {"amount":amount,
            "year": year, 
            "area": area, 
            "duration": duration, 
            "title": title, 
            "subject": subject,
            "population": population, 
            "strategy": strategy, 
            "transaction": transaction,
            "funder": funder,
            "funder_addr": funder_addr, 
            "recipient": recipient, 
            "recipient_addr": recipient_addr,
            "description": desc, 
            "source": source
            
}

funder_df = pd.DataFrame(data= funder)
funder_df.head()

,amount,year,area,duration,title,subject,population,strategy,transaction,funder,funder_addr,recipient,recipient_addr,description,source
0,"TOTAL AMOUNT: $2,250,000",FISCAL YEAR: 2024,PROGRAM AREA: National Program,GRANT DURATION: 1 year,GRANT TITLE: National Center for Civic Innovation,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Not Specified,SUPPORT STRATEGY: Capacity-building and techni...,TRANSACTION TYPE: Cash grants,Carnegie Corporation of New York,"New York City, NY United States",NATIONAL CENTER FOR CIVIC INNOVATION INC,"New York City, NY United States",For core support of OpenSciEd,http://foundationcenter.org/gain-knowledge/fou...
1,"TOTAL AMOUNT: $2,100,000",FISCAL YEAR: 2024,PROGRAM AREA: Global Policy and Advocacy,GRANT DURATION: 1 year,GRANT TITLE: The Asahi Shimbun Company,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Not Specified,SUPPORT STRATEGY: Program support,TRANSACTION TYPE: Cash grants,BILL & MELINDA GATES FOUNDATION,"Seattle, WA United States",The Asahi Shimbun Company,,to support Asahi Shimbun coverage of developme...,http://foundationcenter.org/gain-knowledge/fou...
2,"TOTAL AMOUNT: $2,100,000",FISCAL YEAR: 2024,PROGRAM AREA: Journalism,GRANT DURATION: 1 year,,SUBJECT:Media Content and Platforms; Media Con...,POPULATION SERVED: Adults; Self-employed peopl...,SUPPORT STRATEGY: Recordings and broadcasts; O...,TRANSACTION TYPE: Cash grants,"John S. and James L. Knight Foundation, Inc.","Miami, FL United States","The Video Consortium, Inc.","Mt Kisco, NY United States",To launch templatized local nonfiction video h...,https://knightfoundation.org/grants/
3,"TOTAL AMOUNT: $2,000,000",FISCAL YEAR: 2024,PROGRAM AREA: Life Sciences,GRANT DURATION: 1 year,,SUBJECT:Media Content and Platforms; Media Con...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: Presentations and productions,TRANSACTION TYPE: Cash grants,John Templeton Foundation,"Conshohocken, PA United States",National Geographic Society,"Washington, DC United States",The National Geographic Society will support a...,https://www.templeton.org/grants/grant-database
4,"TOTAL AMOUNT: $2,000,000",FISCAL YEAR: 2024,,GRANT DURATION: 1 year,,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Mackenzie Scott,"Seattle, WA United States",Local Media Foundation,"Lake City, MI United States",To ensure a healthy future for local journalis...,https://yieldgiving.com/gifts


Clean Data

In [ ]:
# Drop duplicates
funder_df_1 = funder_df.drop_duplicates(keep="first")
len(funder_df_1)

# Clean text
funder_df_1.amount = funder_df_1.amount.str.replace('TOTAL AMOUNT:', '')
funder_df_1.year = funder_df_1.year.str.replace('FISCAL YEAR:', '')
funder_df_1.area = funder_df_1.area.str.replace('PROGRAM AREA:', '')
funder_df_1.duration = funder_df_1.duration.str.replace('GRANT DURATION:', '')
funder_df_1.title = funder_df_1.title.str.replace('GRANT TITLE:', '')
funder_df_1.subject = funder_df_1.subject.str.replace('SUBJECT:', '')
funder_df_1.population = funder_df_1.population.str.replace('POPULATION SERVED: ', '')
funder_df_1.strategy = funder_df_1.strategy.str.replace('SUPPORT STRATEGY:', '')
funder_df_1.transaction = funder_df_1.transaction.str.replace('TRANSACTION TYPE: ', '')

# Remove space
funder_df_1 = funder_df_1.applymap(lambda x: x.strip() if isinstance(x, str) else x)



/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_10893/300507869.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funder_df_1.amount = funder_df_1.amount.str.replace('TOTAL AMOUNT:', '')


In [ ]:
funder_df_1.tail()

/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_10893/3978837257.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  funder_df_1 = funder_df_1.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,amount,year,area,duration,title,subject,population,strategy,transaction,funder,funder_addr,recipient,recipient_addr,description,source
268,$250,2024,,1 year,,"Journalism, news and information; Investigativ...",Not Specified,Advocacy; Ethics and accountability; Systems r...,Cash grants,Greater Washington Community Foundation,"Washington, DC United States","PRO Publica, Inc.","New York City, NY United States",Not Specified,http://foundationcenter.org/gain-knowledge/fou...
269,$250,2024,,1 year,,Media Content and Platforms; Radio; Journalism...,Ethnic and racial groups; Sexual identity; Vic...,General support,Cash grants,James & Alvina Bartos Balog Foundation Inc,"Morristown, NJ United States",BOULDER COMMUNITY BROADCAST ASSOCIATION INCORP...,"Boulder, CO United States",UNRESTRICTED,http://foundationcenter.org/gain-knowledge/fou...
270,$250,2024,Advancing The Arts,1 year,,"Journalism, news and information; Investigativ...",Not Specified,Not Specified,Cash grants,The Community Foundation of Western North Caro...,"Asheville, NC United States",Carolina Public Press Inc,"Asheville, NC United States",Not Specified,http://foundationcenter.org/gain-knowledge/fou...
271,$200,2024,,1 year,,Media Content and Platforms; Radio; Journalism...,Adults,Not Specified,Cash grants,Larry and Beatrice Ching Foundation,"Honolulu, HI United States",Hawaii Public Radio,"Honolulu, HI United States",EDUCATIONAL PROGRAMS,http://foundationcenter.org/gain-knowledge/fou...
272,$120,2024,,1 year,,Media Content and Platforms; Media Content and...,Not Specified,Not Specified,Cash grants,The Hollander-Urbach Family Foundation Inc,"Roswell, GA United States",Georgia Public Telecommunications Commission,"Atlanta, GA United States",EDUCATIONAL,http://foundationcenter.org/gain-knowledge/fou...


In [43]:
# Save it locally
funder_df_1.to_csv('grants_2024_clean.csv', index=False)